# 🌦️ Laboratorio 2: Descifrando los Patrones del Clima con Pandas 🐼

## 🌎 Lectura de datos s iniciación con la librería Pandas

---

### 🚀 Bienvenidos al laboratorio de la sesión 2

En la era del Big Data y el cambio climático, **tu capacidad para analizar datos meteorológicos es más crucial que nunca**. Este laboratorio te sumergirá en un océano de datos, donde cada línea de código te acercará a convertirte en un guardián del conocimiento climático.

> "El clima es lo que esperamos, el tiempo es lo que obtenemos." - Mark Twain

### 🌡️ ¿Por qué es Importante?

- **Predicción de eventos extremos**: Sequías, inundaciones, olas de calor.
- **Gestión de recursos hídricos**: Planificación vital para comunidades.
- **Alertas tempranas**: Protección de vidas y propiedades.
- **Comprensión del cambio climático**: Datos que cuentan historias cruciales.

### 🛠️ Tu Kit de Herramientas: Pandas y Datos Reales

Con Pandas, una de las herramientas más poderosas en análisis de datos, explorarás datos meteorológicos reales de Panamá. Cada dataset es una puerta a nuevos descubrimientos.

---

## 🎯 Objetivos del Laboratorio

Al final de esta sesión, serás capaz de:

1. 📊 **Manipular** complejos conjuntos de datos meteorológicos.
2. 🔍 **Identificar** patrones ocultos en variables climáticas.
3. 🧮 **Calcular** estadísticas cruciales para el análisis meteorológico.
4. 📈 **Visualizar** datos climáticos de manera impactante.
5. 🤖 **Desarrollar** modelos para clasificar condiciones climáticas.
6. 📝 **Generar** informes meteorológicos profesionales.

---

## 💪 Competencias que Desarrollarás

| Competencia | Descripción |
|-------------|-------------|
| 🧠 Análisis de Datos | Dominarás la exploración y transformación de datos complejos. |
| 📐 Pensamiento Estadístico | Aplicarás conceptos estadísticos al mundo real del clima. |
| 💻 Programación en Python | Te volverás un experto en Pandas y bibliotecas relacionadas. |
| 🎨 Visualización de Datos | Crearás gráficos que cuentan historias climáticas convincentes. |
| 🌪️ Interpretación Meteorológica | Extraerás insights cruciales de los datos climáticos. |
| 📢 Comunicación Científica | Presentarás hallazgos de manera clara y persuasiva. |
| 🔬 Pensamiento Crítico | Evaluarás datos, identificarás anomalías y formularás hipótesis. |
| 🧩 Resolución de Problemas | Abordarás desafíos complejos con soluciones creativas. |
| ⏱️ Gestión del Tiempo | Manejarás múltiples análisis con eficiencia. |
| 📚 Aprendizaje Continuo | Te mantendrás al día con las últimas técnicas en análisis climático. |

---

### 🌟 El Futuro está en tus Manos

> "Los datos son el nuevo petróleo. Es valioso, pero si no se refina, no se puede usar realmente." - Clive Humby

Prepárate para sumergirte en un mundo donde los datos cobran vida. Cada línea de código que escribas te acercará más a descifrar los misterios del clima y a hacer un impacto real en el mundo.

**¿Estás listo para convertir datos en conocimiento y ese conocimiento en acción?** 🌍🔬🚀

API para conexión

Weather Forecast API
https://open-meteo.com/

Instalación de bibliotecas:

In [32]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 649.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.6 MB/s eta 0:00:00


Este código realiza los siguientes pasos:

- Configuración del cliente Open-Meteo: Configura el cliente con caché y reintento en caso de error.
- Definición de coordenadas: Usa las coordenadas de Panamá.
- Definición de parámetros: Define los parámetros meteorológicos requeridos.
- Realización de la solicitud: Realiza la solicitud a la API de Open-Meteo.
- Procesamiento de la respuesta: Extrae y muestra información relevante de la respuesta.
- Procesamiento de datos horarios: Extrae los datos horarios y los organiza en un DataFrame de pandas.

Es importante destacar que el código de ejecución lo provee directamente la página, lo único que le incluí fueron las coordenadas para Panamá. Acá el link: https://open-meteo.com/en/docs#hourly=temperature_2m,relative_humidity_2m,precipitation_probability,rain,surface_pressure,visibility

In [34]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Coordenadas para Panamá
latitude = 8.9824
longitude = -79.5199

# Variables meteorológicas requeridas
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "hourly": ["temperature_2m", "relative_humidity_2m", "precipitation_probability", "rain", "surface_pressure", "visibility"]
}

# Realizar la solicitud a la API
responses = openmeteo.weather_api(url, params=params)

# Procesar la respuesta de la primera ubicación
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Procesar datos horarios
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation_probability = hourly.Variables(2).ValuesAsNumpy()
hourly_rain = hourly.Variables(3).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(4).ValuesAsNumpy()
hourly_visibility = hourly.Variables(5).ValuesAsNumpy()

# Crear un DataFrame con los datos horarios
hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature_2m,
    "relative_humidity_2m": hourly_relative_humidity_2m,
    "precipitation_probability": hourly_precipitation_probability,
    "rain": hourly_rain,
    "surface_pressure": hourly_surface_pressure,
    "visibility": hourly_visibility
}

hourly_dataframe = pd.DataFrame(data=hourly_data)
print(hourly_dataframe)


Coordinates 9.0°N -79.5°E
Elevation 14.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                         date  temperature_2m  relative_humidity_2m  \
0   2024-06-24 00:00:00+00:00       27.462500                  84.0   
1   2024-06-24 01:00:00+00:00       27.162500                  86.0   
2   2024-06-24 02:00:00+00:00       27.062500                  87.0   
3   2024-06-24 03:00:00+00:00       27.012501                  88.0   
4   2024-06-24 04:00:00+00:00       26.862501                  88.0   
..                        ...             ...                   ...   
163 2024-06-30 19:00:00+00:00       31.362501                  65.0   
164 2024-06-30 20:00:00+00:00       30.312500                  70.0   
165 2024-06-30 21:00:00+00:00       29.462500                  75.0   
166 2024-06-30 22:00:00+00:00       29.012501                  77.0   
167 2024-06-30 23:00:00+00:00       28.762501                  79.0   

     precipitation_probability  rain  surface_pr

# Ejercicios de aplicación

### Ejercicio 1: Exploración básica de datos
- Muestra las primeras 5 filas del DataFrame.
- Imprime información básica del DataFrame (tipos de datos, no nulos, etc.).
- Calcula y muestra estadísticas descriptivas básicas para todas las columnas numéricas.



### Ejercicio 2: Análisis de temperatura
+ Encuentra la temperatura máxima y mínima en el período de pronóstico.
+ Calcula la temperatura promedio para cada día.
+ Identifica las horas del día con temperaturas superiores a 30°C.




### Ejercicio 3: Probabilidad de precipitación
- Calcula la probabilidad promedio de precipitación para cada día.
- Encuentra el día con la mayor probabilidad de precipitación.
- Crea un nuevo DataFrame que solo incluya las horas con probabilidad de precipitación mayor al 50%.



### Ejercicio 4: Análisis de visibilidad
- Calcula la visibilidad promedio por día.
- Identifica los períodos de baja visibilidad (menos de 1000 metros).
- Crea un gráfico de líneas que muestre la variación de la visibilidad a lo largo del tiempo.




### Ejercicio 5: Relación entre variables
- Calcula la correlación entre la temperatura y la humedad relativa.
- Crea un gráfico de dispersión de temperatura vs. humedad relativa.
- Analiza cómo cambia la presión superficial con respecto a la probabilidad de precipitación.




### Ejercicio 6: Análisis avanzado
- Identifica patrones diurnos en la temperatura (comparación día/noche).
- Calcula la amplitud térmica diaria (diferencia entre temperatura máxima y mínima) para cada día.
- Crea una función que clasifique cada hora en una categoría de comfort basada en temperatura y humedad.(por ejemplo: frío, confortable, caluroso)




### Ejercicio 7: Visualización de datos
- Crea un gráfico de líneas que muestre la temperatura, humedad y probabilidad de precipitación en el mismo gráfico.
- Genera un heatmap que muestre la correlación entre todas las variables.
- Crea un gráfico de barras que muestre la precipitación total esperada por día.




### Ejercicio 8: Preparación de informe meteorológico
Crea una función que genere un resumen diario del pronóstico, incluyendo:
- Temperatura máxima y mínima
- Probabilidad de lluvia
- Visibilidad promedio
- Recomendación general basada en las condiciones (por ejemplo, "Buen día para actividades al aire libre")

# Solución

In [ ]:
import pandas as pd

In [ ]:

# Ejercicio 1: Exploración básica de datos
print("Ejercicio 1: Exploración básica de datos")

# a) Muestra las primeras 5 filas del DataFrame
print(hourly_dataframe.head())

# b) Imprime información básica del DataFrame
print(hourly_dataframe.info())

# c) Calcula y muestra estadísticas descriptivas básicas
print(hourly_dataframe.describe())


In [ ]:
# Ejercicio 2: Análisis de temperatura
print("\nEjercicio 2: Análisis de temperatura")

# a) Encuentra la temperatura máxima y mínima
temp_max = hourly_dataframe['temperature_2m'].max()
temp_min = hourly_dataframe['temperature_2m'].min()
print(f"Temperatura máxima: {temp_max}°C, Temperatura mínima: {temp_min}°C")

# b) Calcula la temperatura promedio para cada día
hourly_dataframe['date'] = pd.to_datetime(hourly_dataframe['date'])
daily_avg_temp = hourly_dataframe.groupby(hourly_dataframe['date'].dt.date)['temperature_2m'].mean()
print("Temperatura promedio diaria:")
print(daily_avg_temp)

# c) Identifica las horas con temperaturas superiores a 30°C
hot_hours = hourly_dataframe[hourly_dataframe['temperature_2m'] > 30]
print("Horas con temperatura superior a 30°C:")
print(hot_hours[['date', 'temperature_2m']])

In [ ]:

# Ejercicio 3: Probabilidad de precipitación
print("\nEjercicio 3: Probabilidad de precipitación")

# a) Calcula la probabilidad promedio de precipitación para cada día
daily_precip_prob = hourly_dataframe.groupby(hourly_dataframe['date'].dt.date)['precipitation_probability'].mean()
print("Probabilidad promedio de precipitación diaria:")
print(daily_precip_prob)

# b) Encuentra el día con la mayor probabilidad de precipitación
max_precip_day = daily_precip_prob.idxmax()
print(f"Día con mayor probabilidad de precipitación: {max_precip_day}")

# c) Crea un nuevo DataFrame con probabilidad de precipitación > 50%
high_precip_prob = hourly_dataframe[hourly_dataframe['precipitation_probability'] > 50]
print("Horas con probabilidad de precipitación > 50%:")
print(high_precip_prob[['date', 'precipitation_probability']])



In [ ]:
# Ejercicio 4: Análisis de visibilidad
print("\nEjercicio 4: Análisis de visibilidad")

# a) Calcula la visibilidad promedio por día
daily_avg_visibility = hourly_dataframe.groupby(hourly_dataframe['date'].dt.date)['visibility'].mean()
print("Visibilidad promedio diaria:")
print(daily_avg_visibility)

# b) Identifica los períodos de baja visibilidad (menos de 1000 metros)
low_visibility = hourly_dataframe[hourly_dataframe['visibility'] < 1000]
print("Períodos de baja visibilidad:")
print(low_visibility[['date', 'visibility']])

# c) Variación de la visibilidad a lo largo del tiempo (usando Pandas para crear un gráfico)
visibility_plot = hourly_dataframe.plot(x='date', y='visibility', title='Visibilidad a lo largo del tiempo')
visibility_plot.figure.savefig('visibility_plot.png')
print("Gráfico de visibilidad guardado como 'visibility_plot.png'")



In [ ]:
# Ejercicio 5: Relación entre variables
print("\nEjercicio 5: Relación entre variables")

# a) Calcula la correlación entre la temperatura y la humedad relativa
correlation = hourly_dataframe['temperature_2m'].corr(hourly_dataframe['relative_humidity_2m'])
print(f"Correlación entre temperatura y humedad relativa: {correlation}")

# b) Gráfico de dispersión de temperatura vs. humedad relativa
scatter_plot = hourly_dataframe.plot.scatter(x='temperature_2m', y='relative_humidity_2m', title='Temperatura vs. Humedad Relativa')
scatter_plot.figure.savefig('temp_humidity_scatter.png')
print("Gráfico de dispersión guardado como 'temp_humidity_scatter.png'")

# c) Analiza cómo cambia la presión superficial con respecto a la probabilidad de precipitación
pressure_precip_corr = hourly_dataframe['surface_pressure'].corr(hourly_dataframe['precipitation_probability'])
print(f"Correlación entre presión superficial y probabilidad de precipitación: {pressure_precip_corr}")



In [ ]:
# Ejercicio 6: Análisis avanzado
print("\nEjercicio 6: Análisis avanzado")

# a) Identifica patrones diurnos en la temperatura
hourly_dataframe['hour'] = hourly_dataframe['date'].dt.hour
hourly_temp_pattern = hourly_dataframe.groupby('hour')['temperature_2m'].mean()
print("Patrón diurno de temperatura:")
print(hourly_temp_pattern)

# b) Calcula la amplitud térmica diaria
daily_temp_range = hourly_dataframe.groupby(hourly_dataframe['date'].dt.date)['temperature_2m'].agg(['max', 'min'])
daily_temp_range['amplitude'] = daily_temp_range['max'] - daily_temp_range['min']
print("Amplitud térmica diaria:")
print(daily_temp_range['amplitude'])

# c) Función para clasificar el comfort basado en temperatura y humedad
def classify_comfort(row):
    if row['temperature_2m'] < 18:
        return 'Frío'
    elif row['temperature_2m'] > 28 or row['relative_humidity_2m'] > 70:
        return 'Caluroso'
    else:
        return 'Confortable'

hourly_dataframe['comfort'] = hourly_dataframe.apply(classify_comfort, axis=1)
comfort_counts = hourly_dataframe['comfort'].value_counts()
print("Clasificación de comfort:")
print(comfort_counts)



In [ ]:
# Ejercicio 7: Visualización de datos (usando solo Pandas para gráficos)
print("\nEjercicio 7: Visualización de datos")

# a) Gráfico de líneas para temperatura, humedad y probabilidad de precipitación
multi_line_plot = hourly_dataframe.plot(x='date', y=['temperature_2m', 'relative_humidity_2m', 'precipitation_probability'], title='Condiciones Meteorológicas')
multi_line_plot.figure.savefig('weather_conditions_plot.png')
print("Gráfico de condiciones meteorológicas guardado como 'weather_conditions_plot.png'")

# b) Matriz de correlación (en lugar de heatmap)
correlation_matrix = hourly_dataframe[['temperature_2m', 'relative_humidity_2m', 'precipitation_probability', 'rain', 'surface_pressure', 'visibility']].corr()
print("Matriz de correlación:")
print(correlation_matrix)

# c) Gráfico de barras para precipitación total esperada por día
daily_rain = hourly_dataframe.groupby(hourly_dataframe['date'].dt.date)['rain'].sum()
rain_bar_plot = daily_rain.plot(kind='bar', title='Precipitación Total Diaria')
rain_bar_plot.figure.savefig('daily_rain_plot.png')
print("Gráfico de precipitación diaria guardado como 'daily_rain_plot.png'")



In [ ]:
# Ejercicio 8: Preparación de informe meteorológico
print("\nEjercicio 8: Preparación de informe meteorológico")

def generate_daily_report(date):
    daily_data = hourly_dataframe[hourly_dataframe['date'].dt.date == date]
    max_temp = daily_data['temperature_2m'].max()
    min_temp = daily_data['temperature_2m'].min()
    rain_prob = daily_data['precipitation_probability'].mean()
    avg_visibility = daily_data['visibility'].mean()

    recommendation = "Buen día para actividades al aire libre" if rain_prob < 30 and avg_visibility > 5000 else "Se recomienda precaución en actividades al aire libre"

    report = f"""
    Informe meteorológico para {date}:
    Temperatura máxima: {max_temp:.1f}°C
    Temperatura mínima: {min_temp:.1f}°C
    Probabilidad de lluvia: {rain_prob:.1f}%
    Visibilidad promedio: {avg_visibility:.0f} metros
    Recomendación: {recommendation}
    """
    return report

# Generar informe para el primer día en los datos
first_day = hourly_dataframe['date'].dt.date.min()
print(generate_daily_report(first_day))